# 1.ライブラリの読み込み
上から順に
* グラフを書くためにmatplotlib
* データの整理のためにpandas
* 数学的な計算のためにnumpy
* 時系列データを扱うためにdatetime
* matplotlibでdatetimeのフォーマットを変えるためにmdates

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.dates as mdates
from scipy import signal

# 2.センサーデータの読みこみと確認
ファイルの読み込みに使用した変数名の命名則については以下の通りである．

例　セッション1のAさんのセンサデータ →　data2_c
　　セッション1のAさんのスケーリングデータ　→　data2_c_label_50

In [2]:
%time
#sensor_dataの読み込み

# data2_c = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-A胸-20201111-121953632-ags.csv', encoding="shift-jis")
# data1_b = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-B胸-20201111-122231707-ags.csv', encoding="shift-jis")
# data2_c = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-C胸-20201111-121953885-ags.csv', encoding="shift-jis")
# data1_d = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-D胸-20201111-121954011-ags.csv', encoding="shift-jis")
# data2_c = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-A胸-20201111-145759193-ags.csv', encoding="shift-jis")
# data2_b = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-B胸-20201111-145733362-ags.csv', encoding="shift-jis")
data2_c = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-C胸-20201111-150018208-ags.csv', encoding="shift-jis")
# data2_d = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-D胸-20201111-145806707-ags.csv', encoding="shift-jis")

#emotion_labelの読み込み

#data2_c_label_50 = pd.read_csv('emotion_label/C/emotion_C_20201111_1_50.csv')
#data2_c_label_60 = pd.read_csv('emotion_label/C/emotion_C_20201111_1_60.csv')
#data2_c_label_70 = pd.read_csv('emotion_label/C/emotion_C_20201111_1_70.csv')
#data2_c_label_80 = pd.read_csv('emotion_label/C/emotion_C_20201111_1_80.csv')

data2_c_label_50 = pd.read_csv('emotion_label/session2/C/emotion_C_20201111_2_50.csv')
data2_c_label_60 = pd.read_csv('emotion_label/session2/C/emotion_C_20201111_2_60.csv')
data2_c_label_70 = pd.read_csv('emotion_label/session2/C/emotion_C_20201111_2_70.csv')
data2_c_label_80 = pd.read_csv('emotion_label/session2/C/emotion_C_20201111_2_80.csv')

Wall time: 0 ns


## 2.1 TSND151加速度，角速度センサデータとスケーリングデータの中身
今回のセンサデータの列はセンサの種類，時間，加速度のx，y，z，角速度x，y，zで構成されており，加速度の単位は0.1mG，角速度の単位は0.01dpsである．

スケーリングデータの列は，レコーディング動画の経過時間，感情のレベル値，発話されたユーザ名となっている．emotion_levelの感情値については今回は以下のように考えた．

-4〜-1→negative  0→neutral  １〜4→positive

In [3]:
data2_c_label_50

,start_time,finish_time,emotion_level
0,3:40,3:44,2
1,3:41,3:45,2
2,3:42,3:46,2
3,3:43,3:47,2
4,3:44,3:48,2
...,...,...,...
723,18:04,18:08,1
724,18:05,18:09,1
725,18:06,18:10,1
726,18:07,18:11,1


In [4]:
#欠損値削除
def dropna(label):
    a = label.dropna(how='all', axis=1)
    return a

#data2_c_label_50 = dropna(data2_c_label_50)
#data2_c_label_60 = dropna(data2_c_label_60)
#data2_c_label_70 = dropna(data2_c_label_70)
#data2_c_label_80 = dropna(data2_c_label_80)

data2_c_label_50 = dropna(data2_c_label_50)
data2_c_label_60 = dropna(data2_c_label_60)
data2_c_label_70 = dropna(data2_c_label_70)
data2_c_label_80 = dropna(data2_c_label_80)

In [5]:
#欠損値削除
def dropna(label):
    a = label.dropna(how='all', axis=0)
    return a

#data2_c_label_50 = dropna(data2_c_label_50)
#data2_c_label_60 = dropna(data2_c_label_60)
#data2_c_label_70 = dropna(data2_c_label_70)
#data2_c_label_80 = dropna(data2_c_label_80)

data2_c_label_50 = dropna(data2_c_label_50)
data2_c_label_60 = dropna(data2_c_label_60)
data2_c_label_70 = dropna(data2_c_label_70)
data2_c_label_80 = dropna(data2_c_label_80)

In [6]:
data2_c_label_50

,start_time,finish_time,emotion_level
0,3:40,3:44,2
1,3:41,3:45,2
2,3:42,3:46,2
3,3:43,3:47,2
4,3:44,3:48,2
...,...,...,...
723,18:04,18:08,1
724,18:05,18:09,1
725,18:06,18:10,1
726,18:07,18:11,1


In [7]:
#生データの確認
display(data2_c)
display(data2_c_label_50)

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z
0,ags,15:00:18.260,9805,-1881,2219,437,354,149
1,ags,15:00:18.265,9810,-1893,2336,431,348,167
2,ags,15:00:18.270,9795,-1884,2490,589,305,155
3,ags,15:00:18.275,9829,-1705,2581,705,287,167
4,ags,15:00:18.280,9763,-1705,2600,809,262,112
...,...,...,...,...,...,...,...,...
444281,ags,15:37:19.665,1279,-1803,9121,2376,232,1381
444282,ags,15:37:19.670,1042,-2084,9497,2449,171,1387
444283,ags,15:37:19.675,891,-2306,9814,2431,-5,1295
444284,ags,15:37:19.680,815,-2323,9924,2424,-230,1082


,start_time,finish_time,emotion_level
0,3:40,3:44,2
1,3:41,3:45,2
2,3:42,3:46,2
3,3:43,3:47,2
4,3:44,3:48,2
...,...,...,...
723,18:04,18:08,1
724,18:05,18:09,1
725,18:06,18:10,1
726,18:07,18:11,1


In [8]:
#加速度削除
data2_c = data2_c.drop(["加速度X", "加速度Y", "加速度Z"],axis=1)

In [9]:
#合成加速度と合成角速度の算出
def norm_order(sensor_data):
    #sensor_data["合成加速度"] = np.sqrt(sensor_data["加速度X"]*sensor_data["加速度X"] + sensor_data["加速度Y"]*sensor_data["加速度Y"] + sensor_data["加速度Z"]*sensor_data["加速度Z"])
    sensor_data["合成角速度"] = np.sqrt(sensor_data["角速度X"]*sensor_data["角速度X"] + sensor_data["角速度Y"]*sensor_data["角速度Y"] + sensor_data["角速度Z"]*sensor_data["角速度Z"])
    
# norm_order(data2_c)
# norm_order(data1_b)
# norm_order(data2_c)
# norm_order(data1_d)
# norm_order(data2_c)
# norm_order(data2_b)
norm_order(data2_c)
# norm_order(data2_d)

In [10]:
data2_c

,種別,時刻,角速度X,角速度Y,角速度Z,合成角速度
0,ags,15:00:18.260,437,354,149,581.795497
1,ags,15:00:18.265,431,348,167,578.579294
2,ags,15:00:18.270,589,305,155,681.154168
3,ags,15:00:18.275,705,287,167,779.283645
4,ags,15:00:18.280,809,262,112,857.711490
...,...,...,...,...,...,...
444281,ags,15:37:19.665,2376,232,1381,2757.963198
444282,ags,15:37:19.670,2449,171,1387,2819.682784
444283,ags,15:37:19.675,2431,-5,1295,2754.416635
444284,ags,15:37:19.680,2424,-230,1082,2664.469929


# 3.スケーリングデータに即したセンサデータの抽出
スケーリングデータに即したセンサデータの取得方法は以下の通りである．
1. センサデータ及びスケーリングデータの時刻列をdatetime化
2. timedelta関数を使用して，時刻計算を行い，実験実施日に合わせていく
3. センサデータの時刻列をインデックス化

  時刻列をインデックスにすることで，時間を使ってのデータのスライスが可能となる

4. スケーリングデータの時刻列を文字列型のリストにする
5. ４の操作を行うことでセンサデータからスケーリングデータに即したデータを抽出するのが容易になる

In [11]:
#センサデータの時刻合わせ
def datetime_order(sensor_data):
    sensor_data['時刻'] = pd.to_datetime(sensor_data['時刻'], format='%H:%M:%S.%f')
    sensor_data['時刻'] = sensor_data['時刻'] + dt.timedelta(days=44144)

# datetime_order(data2_c)
# datetime_order(data1_b)
# datetime_order(data2_c)
# datetime_order(data1_d)

# datetime_order(data2_c)
# datetime_order(data2_b)
datetime_order(data2_c)
# datetime_order(data2_d)

In [12]:
data2_c

,種別,時刻,角速度X,角速度Y,角速度Z,合成角速度
0,ags,2020-11-11 15:00:18.260,437,354,149,581.795497
1,ags,2020-11-11 15:00:18.265,431,348,167,578.579294
2,ags,2020-11-11 15:00:18.270,589,305,155,681.154168
3,ags,2020-11-11 15:00:18.275,705,287,167,779.283645
4,ags,2020-11-11 15:00:18.280,809,262,112,857.711490
...,...,...,...,...,...,...
444281,ags,2020-11-11 15:37:19.665,2376,232,1381,2757.963198
444282,ags,2020-11-11 15:37:19.670,2449,171,1387,2819.682784
444283,ags,2020-11-11 15:37:19.675,2431,-5,1295,2754.416635
444284,ags,2020-11-11 15:37:19.680,2424,-230,1082,2664.469929


In [13]:
#data2_c_label_50['start_time'] = pd.to_datetime(data2_c_label_50['start_time'], format='%M:%S')
#data2_c_label_50['finish_time'] = pd.to_datetime(data2_c_label_50['finish_time'], format='%M:%S')

In [14]:
#data2_c_label_60['start_time'] = pd.to_datetime(data2_c_label_60['start_time'], format='%M:%S')
#data2_c_label_60['finish_time'] = pd.to_datetime(data2_c_label_60['finish_time'], format='%M:%S')

In [15]:
#data2_c_label_70['start_time'] = pd.to_datetime(data2_c_label_70['start_time'], format='%M:%S')
#data2_c_label_70['finish_time'] = pd.to_datetime(data2_c_label_70['finish_time'], format='%M:%S')

In [16]:
#data2_c_label_80['start_time'] = pd.to_datetime(data2_c_label_80['start_time'], format='%M:%S')
#data2_c_label_80['finish_time'] = pd.to_datetime(data2_c_label_80['finish_time'], format='%M:%S')

In [17]:
data2_c_label_50['start_time'] = pd.to_datetime(data2_c_label_50['start_time'], format='%M:%S')
data2_c_label_50['finish_time'] = pd.to_datetime(data2_c_label_50['finish_time'], format='%M:%S')

In [18]:
data2_c_label_60['start_time'] = pd.to_datetime(data2_c_label_60['start_time'], format='%M:%S')
data2_c_label_60['finish_time'] = pd.to_datetime(data2_c_label_60['finish_time'], format='%M:%S')

In [19]:
data2_c_label_70['start_time'] = pd.to_datetime(data2_c_label_70['start_time'], format='%M:%S')
data2_c_label_70['finish_time'] = pd.to_datetime(data2_c_label_70['finish_time'], format='%M:%S')

In [20]:
data2_c_label_80['start_time'] = pd.to_datetime(data2_c_label_80['start_time'], format='%M:%S')
data2_c_label_80['finish_time'] = pd.to_datetime(data2_c_label_80['finish_time'], format='%M:%S')

In [21]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_c)
display(data2_c_label_50)

,種別,時刻,角速度X,角速度Y,角速度Z,合成角速度
0,ags,2020-11-11 15:00:18.260,437,354,149,581.795497
1,ags,2020-11-11 15:00:18.265,431,348,167,578.579294
2,ags,2020-11-11 15:00:18.270,589,305,155,681.154168
3,ags,2020-11-11 15:00:18.275,705,287,167,779.283645
4,ags,2020-11-11 15:00:18.280,809,262,112,857.711490
...,...,...,...,...,...,...
444281,ags,2020-11-11 15:37:19.665,2376,232,1381,2757.963198
444282,ags,2020-11-11 15:37:19.670,2449,171,1387,2819.682784
444283,ags,2020-11-11 15:37:19.675,2431,-5,1295,2754.416635
444284,ags,2020-11-11 15:37:19.680,2424,-230,1082,2664.469929


,start_time,finish_time,emotion_level
0,1900-01-01 00:03:40,1900-01-01 00:03:44,2
1,1900-01-01 00:03:41,1900-01-01 00:03:45,2
2,1900-01-01 00:03:42,1900-01-01 00:03:46,2
3,1900-01-01 00:03:43,1900-01-01 00:03:47,2
4,1900-01-01 00:03:44,1900-01-01 00:03:48,2
...,...,...,...
723,1900-01-01 00:18:04,1900-01-01 00:18:08,1
724,1900-01-01 00:18:05,1900-01-01 00:18:09,1
725,1900-01-01 00:18:06,1900-01-01 00:18:10,1
726,1900-01-01 00:18:07,1900-01-01 00:18:11,1


### 補足(実験実施時間)

本実験は，zoomレコーディング動画の経過時間でスケーリングデータのemotion_levelをつけたため動画の開始時刻を使って実際の実験実施時間と合わせる必要がある．

方法としては，経過時間と動画の開始時刻を足して実験実施時間を算出する．

動画の開始時間↓

第1セッション12時34分30秒開始

第2セッション15時11分44秒開始

In [22]:
#data2_c_label_50['start_time'] = data2_c_label_50['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
#data2_c_label_50['finish_time'] = data2_c_label_50['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [23]:
#data2_c_label_60['start_time'] = data2_c_label_60['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
#data2_c_label_60['finish_time'] = data2_c_label_60['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [24]:
#data2_c_label_70['start_time'] = data2_c_label_70['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
#data2_c_label_70['finish_time'] = data2_c_label_70['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [25]:
#data2_c_label_80['start_time'] = data2_c_label_80['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
#data2_c_label_80['finish_time'] = data2_c_label_80['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [26]:
data2_c_label_50['start_time'] = data2_c_label_50['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_c_label_50['finish_time'] = data2_c_label_50['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [27]:
data2_c_label_60['start_time'] = data2_c_label_60['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_c_label_60['finish_time'] = data2_c_label_60['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [28]:
data2_c_label_70['start_time'] = data2_c_label_70['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_c_label_70['finish_time'] = data2_c_label_70['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [29]:
data2_c_label_80['start_time'] = data2_c_label_80['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_c_label_80['finish_time'] = data2_c_label_80['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [30]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_c_label_80)

,start_time,finish_time,emotion_level
0,2020-11-11 15:15:25,2020-11-11 15:15:29,2
1,2020-11-11 15:15:26,2020-11-11 15:15:30,2
2,2020-11-11 15:15:27,2020-11-11 15:15:31,2
3,2020-11-11 15:15:28,2020-11-11 15:15:32,2
4,2020-11-11 15:15:29,2020-11-11 15:15:33,2
...,...,...,...
683,2020-11-11 15:29:48,2020-11-11 15:29:52,1
684,2020-11-11 15:29:49,2020-11-11 15:29:53,1
685,2020-11-11 15:29:50,2020-11-11 15:29:54,1
686,2020-11-11 15:29:51,2020-11-11 15:29:55,1


In [31]:
#datetime化したラベルデータを保存
# data2_c_label_50.to_csv('emotion_label/label_datetime/emotion_A_20201111_1_datetime.csv', index=False)
# data2_c_label_60.to_csv('emotion_label/label_datetime/emotion_B_20201111_1_datetime.csv', index=False)
# data2_c_label_70.to_csv('emotion_label/label_datetime/emotion_C_20201111_1_datetime.csv', index=False)
# data2_c_label_80.to_csv('emotion_label/label_datetime/emotion_D_20201111_1_datetime.csv', index=False)

# data2_c_label_50.to_csv('emotion_label/label_datetime/emotion_A_20201111_2_datetime.csv', index=False)
# data2_c_label_60.to_csv('emotion_label/label_datetime/emotion_B_20201111_2_datetime.csv', index=False)
# data2_c_label_70.to_csv('emotion_label/label_datetime/emotion_C_20201111_2_datetime.csv', index=False)
# data2_c_label_80.to_csv('emotion_label/label_datetime/emotion_D_20201111_2_datetime.csv', index=False)

In [32]:
#datetime化した加速度及び角速度データを保存
# data2_c.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-121953632-ags_datetime.csv', index=False)
# data1_b.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-122231707-ags_datetime.csv', index=False)
# data2_c.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-121953885-ags_datetime.csv', index=False)
# data1_d.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-121954011-ags_datetime.csv', index=False)

# data2_c.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-145759193-ags_datetime.csv', index=False)
# data2_b.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-145733362-ags_datetime.csv', index=False)
# data2_c.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-150018208-ags_datetime.csv', index=False)
# data2_d.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-145806707-ags_datetime.csv', index=False)

In [33]:
#時刻をインデックスに
data2_c.set_index("時刻", inplace=True)
# data1_b.set_index("時刻", inplace=True)
# data2_c.set_index("時刻", inplace=True)
# data1_d.set_index("時刻", inplace=True)

# data2_c.set_index("時刻", inplace=True)
# data2_b.set_index("時刻", inplace=True)
# data2_c.set_index("時刻", inplace=True)
# data2_d.set_index("時刻", inplace=True)

In [34]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_c)

,種別,角速度X,角速度Y,角速度Z,合成角速度
時刻,,,,,
2020-11-11 15:00:18.260,ags,437,354,149,581.795497
2020-11-11 15:00:18.265,ags,431,348,167,578.579294
2020-11-11 15:00:18.270,ags,589,305,155,681.154168
2020-11-11 15:00:18.275,ags,705,287,167,779.283645
2020-11-11 15:00:18.280,ags,809,262,112,857.711490
...,...,...,...,...,...
2020-11-11 15:37:19.665,ags,2376,232,1381,2757.963198
2020-11-11 15:37:19.670,ags,2449,171,1387,2819.682784
2020-11-11 15:37:19.675,ags,2431,-5,1295,2754.416635


In [35]:
data2_c_label_50["start_time"] = data2_c_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_c_label_50["finish_time"] = data2_c_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [36]:
data2_c_label_60["start_time"] = data2_c_label_60["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_c_label_60["finish_time"] = data2_c_label_60["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [37]:
data2_c_label_70["start_time"] = data2_c_label_70["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_c_label_70["finish_time"] = data2_c_label_70["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [38]:
data2_c_label_80["start_time"] = data2_c_label_80["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_c_label_80["finish_time"] = data2_c_label_80["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [39]:
# data2_c_label_50["start_time"] = data2_c_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_c_label_50["finish_time"] = data2_c_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [40]:
# data2_c_label_60["start_time"] = data2_c_label_60["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_c_label_60["finish_time"] = data2_c_label_60["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [41]:
# data2_c_label_70["start_time"] = data2_c_label_70["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_c_label_70["finish_time"] = data2_c_label_70["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [42]:
# data2_c_label_80["start_time"] = data2_c_label_80["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_c_label_80["finish_time"] = data2_c_label_80["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [43]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_c_label_70)

,start_time,finish_time,emotion_level
0,2020-11-11 15:15:24,2020-11-11 15:15:28,2
1,2020-11-11 15:15:25,2020-11-11 15:15:29,2
2,2020-11-11 15:15:26,2020-11-11 15:15:30,2
3,2020-11-11 15:15:27,2020-11-11 15:15:31,2
4,2020-11-11 15:15:28,2020-11-11 15:15:32,2
...,...,...,...
698,2020-11-11 15:29:48,2020-11-11 15:29:52,1
699,2020-11-11 15:29:49,2020-11-11 15:29:53,1
700,2020-11-11 15:29:50,2020-11-11 15:29:54,1
701,2020-11-11 15:29:51,2020-11-11 15:29:55,1


In [44]:
#startとfinishをリスト化
data2_c_50_start_time = list(data2_c_label_50["start_time"])
data2_c_50_finish_time = list(data2_c_label_50["finish_time"])

data2_c_60_start_time = list(data2_c_label_60["start_time"])
data2_c_60_finish_time = list(data2_c_label_60["finish_time"])

data2_c_70_start_time = list(data2_c_label_70["start_time"])
data2_c_70_finish_time = list(data2_c_label_70["finish_time"])

data2_c_80_start_time = list(data2_c_label_80["start_time"])
data2_c_80_finish_time = list(data2_c_label_80["finish_time"])

# data2_c_50_start_time = list(data2_c_label_50["start_time"])
# data2_c_50_finish_time = list(data2_c_label_50["finish_time"])

# data2_c_60_start_time = list(data2_c_label_60["start_time"])
# data2_c_60_finish_time = list(data2_c_label_60["finish_time"])

# data2_c_70_start_time = list(data2_c_label_70["start_time"])
# data2_c_70_finish_time = list(data2_c_label_70["finish_time"])

# data2_c_80_start_time = list(data2_c_label_80["start_time"])
# data2_c_80_finish_time = list(data2_c_label_80["finish_time"])

In [45]:
def sensor_segmentation_order(data, data_start_time, data_finish_time, data_label):
    a = [data[data_start_time[i]:data_finish_time[i]]for i in range(0, len(data_label), 1)]
    return a

data2_c_record_1 = sensor_segmentation_order(data2_c, data2_c_50_start_time, data2_c_50_finish_time, data2_c_label_50)
data2_c_record_2 = sensor_segmentation_order(data2_c, data2_c_60_start_time, data2_c_60_finish_time, data2_c_label_60)
data2_c_record_3 = sensor_segmentation_order(data2_c, data2_c_70_start_time, data2_c_70_finish_time, data2_c_label_70)
data2_c_record_4 = sensor_segmentation_order(data2_c, data2_c_80_start_time, data2_c_80_finish_time, data2_c_label_80)

# data2_c_record_1 = sensor_segmentation_order(data2_c, data2_c_50_start_time, data2_c_50_finish_time, data2_c_label_50)
# data2_c_record_2 = sensor_segmentation_order(data2_c, data2_c_60_start_time, data2_c_60_finish_time, data2_c_label_60)
# data2_c_record_3 = sensor_segmentation_order(data2_c, data2_c_70_start_time, data2_c_70_finish_time, data2_c_label_70)
# data2_c_record_4 = sensor_segmentation_order(data2_c, data2_c_80_start_time, data2_c_80_finish_time, data2_c_label_80)

# 4.特徴量算出
感情の識別を行うため学習器作成のために特徴量の算出を行う．今回特徴量としたのは，加速度の各軸及び角速度の各軸における時間領域での平均値，分散値，最大値，最小値及び，周波数領域での特徴量である最大ピーク周波数，最大ピーク値である．プログラムの処理手順を以下に示す．

1. 平均値の算出
2. 分散値の算出
3. 最大値の算出
4. 最小値の算出
5. 最大ピーク周波数及び最大ピーク値の算出
5. 全特徴量の数値を表記したデータフレームを作成

## 4.1 平均値算出
### 4.1.1 加速度

In [46]:
#加速度x,y,z,norm平均値
def acc_gyr_mean_order(data_record, data_label):
#     a = [np.mean(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
#     b = [np.mean(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
#     c = [np.mean(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
#     d = [np.mean(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
#     a = pd.DataFrame(a)
#     b = pd.DataFrame(b)
#     c = pd.DataFrame(c)
#     d = pd.DataFrame(d)
#     acc_mean_list = pd.concat([a, b, c], axis=1, ignore_index=True)
#     acc_mean_list.columns = ["accx_mean", "accy_mean", "accz_mean"]
#     acc_mean_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_mean_list.columns = ["accx_mean", "accy_mean", "accz_mean", "accn_mean"]
    e = [np.mean(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.mean(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.mean(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.mean(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
#     gyr_mean_list = pd.concat([e, f, g], axis=1, ignore_index=True)
#     gyr_mean_list.columns = ["gyrx_mean", "gyry_mean", "gyrz_mean"]
    gyr_mean_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
    gyr_mean_list.columns = ["gyrx_mean", "gyry_mean", "gyrz_mean", "gyrn_mean"]
#    mean_list = pd.concat([acc_mean_list, gyr_mean_list], axis=1, ignore_index=False)
    return gyr_mean_list

data2_c_50_record_mean = acc_gyr_mean_order(data2_c_record_1, data2_c_label_50)
data2_c_60_record_mean = acc_gyr_mean_order(data2_c_record_2, data2_c_label_60)
data2_c_70_record_mean = acc_gyr_mean_order(data2_c_record_3, data2_c_label_70)
data2_c_80_record_mean = acc_gyr_mean_order(data2_c_record_4, data2_c_label_80)

# data2_c_50_record_mean = acc_gyr_mean_order(data2_c_record_1, data2_c_label_50)
# data2_c_60_record_mean = acc_gyr_mean_order(data2_c_record_2, data2_c_label_60)
# data2_c_70_record_mean = acc_gyr_mean_order(data2_c_record_3, data2_c_label_70)
# data2_c_80_record_mean = acc_gyr_mean_order(data2_c_record_4, data2_c_label_80)

# a = data2_c_50_record_mean
# b = data2_c_60_record_mean
# c = data2_c_70_record_mean
# d = data2_c_80_record_mean

# e = data2_c_50_record_mean
# f = data2_c_60_record_mean
# g = data2_c_70_record_mean
# h = data2_c_80_record_mean

# mean_list = pd.concat([a, b, c, d], ignore_index=True)
# mean_list

In [47]:
#加速度x,y,z,norm分散値
def acc_gyr_var_order(data_record, data_label):
#     a = [np.var(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
#     b = [np.var(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
#     c = [np.var(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
#     d = [np.var(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
#     a = pd.DataFrame(a)
#     b = pd.DataFrame(b)
#     c = pd.DataFrame(c)
#     d = pd.DataFrame(d)
#     acc_var_list = pd.concat([a, b, c], axis=1, ignore_index=True)
#     acc_var_list.columns = ["accx_var", "accy_var", "accz_var"]
#     acc_var_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_var_list.columns = ["accx_var", "accy_var", "accz_var", "accn_var"]
    e = [np.var(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.var(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.var(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.var(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
#     gyr_var_list = pd.concat([e, f, g], axis=1, ignore_index=True)
#     gyr_var_list.columns = ["gyrx_var", "gyry_var", "gyrz_var"]
    gyr_var_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
    gyr_var_list.columns = ["gyrx_var", "gyry_var", "gyrz_var", "gyrn_var"]
#    var_list = pd.concat([acc_var_list, gyr_var_list], axis=1, ignore_index=False)
    return gyr_var_list

data2_c_50_record_var = acc_gyr_var_order(data2_c_record_1, data2_c_label_50)
data2_c_60_record_var = acc_gyr_var_order(data2_c_record_2, data2_c_label_60)
data2_c_70_record_var = acc_gyr_var_order(data2_c_record_3, data2_c_label_70)
data2_c_80_record_var = acc_gyr_var_order(data2_c_record_4, data2_c_label_80)

# data2_c_50_record_var = acc_gyr_var_order(data2_c_record_1, data2_c_label_50)
# data2_c_60_record_var = acc_gyr_var_order(data2_c_record_2, data2_c_label_60)
# data2_c_70_record_var = acc_gyr_var_order(data2_c_record_3, data2_c_label_70)
# data2_c_80_record_var = acc_gyr_var_order(data2_c_record_4, data2_c_label_80)

# a = data2_c_50_record_var
# b = data2_c_60_record_var
# c = data2_c_70_record_var
# d = data2_c_80_record_var

# e = data2_c_50_record_var
# f = data2_c_60_record_var
# g = data2_c_70_record_var
# h = data2_c_80_record_var

# var_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# var_list

In [48]:
#加速度x,y,z,norm分散値
def acc_gyr_max_order(data_record, data_label):
#     a = [np.max(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
#     b = [np.max(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
#     c = [np.max(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
#     d = [np.max(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
#     a = pd.DataFrame(a)
#     b = pd.DataFrame(b)
#     c = pd.DataFrame(c)
#     d = pd.DataFrame(d)
#     acc_max_list = pd.concat([a, b, c], axis=1, ignore_index=True)
#     acc_max_list.columns = ["accx_max", "accy_max", "accz_max"]
#     acc_max_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_max_list.columns = ["accx_max", "accy_max", "accz_max", "accn_max"]
    e = [np.max(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.max(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.max(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.max(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
#     gyr_max_list = pd.concat([e, f, g], axis=1, ignore_index=True)
#     gyr_max_list.columns = ["gyrx_max", "gyry_max", "gyrz_max"]
    gyr_max_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
    gyr_max_list.columns = ["gyrx_max", "gyry_max", "gyrz_max", "gyrn_max"]
#    max_list = pd.concat([acc_max_list, gyr_max_list], axis=1, ignore_index=False)
    return gyr_max_list

data2_c_50_record_max = acc_gyr_max_order(data2_c_record_1, data2_c_label_50)
data2_c_60_record_max = acc_gyr_max_order(data2_c_record_2, data2_c_label_60)
data2_c_70_record_max = acc_gyr_max_order(data2_c_record_3, data2_c_label_70)
data2_c_80_record_max = acc_gyr_max_order(data2_c_record_4, data2_c_label_80)

# data2_c_50_record_max = acc_gyr_max_order(data2_c_record_1, data2_c_label_50)
# data2_c_60_record_max = acc_gyr_max_order(data2_c_record_2, data2_c_label_60)
# data2_c_70_record_max = acc_gyr_max_order(data2_c_record_3, data2_c_label_70)
# data2_c_80_record_max = acc_gyr_max_order(data2_c_record_4, data2_c_label_80)

# a = data2_c_50_record_max
# b = data2_c_60_record_max
# c = data2_c_70_record_max
# d = data2_c_80_record_max

# e = data2_c_50_record_max
# f = data2_c_60_record_max
# g = data2_c_70_record_max
# h = data2_c_80_record_max

# max_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# max_list

In [49]:
#加速度x,y,z,norm分散値
def acc_gyr_min_order(data_record, data_label):
#     a = [np.min(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
#     b = [np.min(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
#     c = [np.min(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
#     d = [np.min(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
#     a = pd.DataFrame(a)
#     b = pd.DataFrame(b)
#     c = pd.DataFrame(c)
#     d = pd.DataFrame(d)
#     acc_min_list = pd.concat([a, b, c], axis=1, ignore_index=True)
#     acc_min_list.columns = ["accx_min", "accy_min", "accz_min"]
#     acc_min_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_min_list.columns = ["accx_min", "accy_min", "accz_min", "accn_min"]
    e = [np.min(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.min(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.min(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.min(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
#    gyr_min_list = pd.concat([e, f, g], axis=1, ignore_index=True)
#    gyr_min_list.columns = ["gyrx_min", "gyry_min", "gyrz_min"]
    gyr_min_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
    gyr_min_list.columns = ["gyrx_min", "gyry_min", "gyrz_min", "gyrn_min"]
#    min_list = pd.concat([acc_min_list, gyr_min_list], axis=1, ignore_index=False)
    return gyr_min_list

data2_c_50_record_min = acc_gyr_min_order(data2_c_record_1, data2_c_label_50)
data2_c_60_record_min = acc_gyr_min_order(data2_c_record_2, data2_c_label_60)
data2_c_70_record_min = acc_gyr_min_order(data2_c_record_3, data2_c_label_70)
data2_c_80_record_min = acc_gyr_min_order(data2_c_record_4, data2_c_label_80)

# data2_c_50_record_min = acc_gyr_min_order(data2_c_record_1, data2_c_label_50)
# data2_c_60_record_min = acc_gyr_min_order(data2_b_record_2, data2_c_label_60)
# data2_c_70_record_min = acc_gyr_min_order(data2_c_record_3, data2_c_label_70)
# data2_c_80_record_min = acc_gyr_min_order(data2_d_record_4, data2_c_label_80)

# a = data2_c_50_record_min
# b = data2_c_60_record_min
# c = data2_c_70_record_min
# d = data2_c_80_record_min

# e = data2_c_50_record_min
# f = data2_c_60_record_min
# g = data2_c_70_record_min
# h = data2_c_80_record_min

# min_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# min_list

## 4.5 最大ピーク周波数及び最大ピーク値

In [50]:
def fq_amp_max_order(data_record, data_label):
    #サンプル数とサンプリング周期及びカットする周波数を設定
    N = 512 # サンプル数
    dt = 0.005 # サンプリング周期(sec)
    fc = 100
    t = np.arange(0, N*dt, dt) # 時間軸
    
#     f = [data_record[i]["合成加速度"].head(512) for i in range(0, len(data_label), 1)]

#     # 高速フーリエ変換(FFT)
#     F = np.fft.fft(f)
#     F_abs = np.abs(F) # 複素数を絶対値に変換
#     F_abs_amp = F_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
#     F_abs_amp[0] = F_abs_amp[0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)

#     # 周波数軸のデータ作成
#     fq = np.linspace(0, 1.0/dt, N) # 周波数軸　linspace(開始,終了,分割数)

#     # フィルタリング①（周波数でカット）＊＊＊＊＊＊
#     F2 = np.copy(F) # FFT結果コピー
#     for i in range(0, len(data_record), 1):
#         F2[i][(fq > fc)] = 0
#     F2_abs = np.abs(F2) # FFTの複素数結果を絶対値に変換
#     F2_abs_amp = F2_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
#     for i in range(0, len(data_record), 1):
#         F2_abs_amp[i][0] = F2_abs_amp[i][0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)
    
#     fq_amp = pd.DataFrame(fq)
#     amp = [pd.DataFrame(F2_abs_amp[i])for i in range(0, len(data_label), 1)]
#     a = [pd.concat([fq_amp, amp[i]], axis=1, ignore_index=True)for i in range(0, len(data_label), 1)]

#     a = [pd.DataFrame(a[i]).drop(a[i].index[[0]])for i in range(0, len(data_label), 1)]
#     a = [a[i][a[i][1] == np.max(a[i][1])]for i in range(0, len(data_label), 1)]
#     data_record_accn_fq_amp_max = pd.concat(a, ignore_index=True)
#     data_record_accn_fq_amp_max.columns = ["accn_max_fq", "accn_max_peak"]
    
    f = [data_record[i]["合成角速度"].head(512) for i in range(0, len(data_label), 1)]

    # 高速フーリエ変換(FFT)
    F = np.fft.fft(f)
    F_abs = np.abs(F) # 複素数を絶対値に変換
    F_abs_amp = F_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    F_abs_amp[0] = F_abs_amp[0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)

    # 周波数軸のデータ作成
    fq = np.linspace(0, 1.0/dt, N) # 周波数軸　linspace(開始,終了,分割数)

    # フィルタリング①（周波数でカット）＊＊＊＊＊＊
    F2 = np.copy(F) # FFT結果コピー
    for i in range(0, len(data_record), 1):
        F2[i][(fq > fc)] = 0
    F2_abs = np.abs(F2) # FFTの複素数結果を絶対値に変換
    F2_abs_amp = F2_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    for i in range(0, len(data_record), 1):
        F2_abs_amp[i][0] = F2_abs_amp[i][0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)
    
    fq_amp = pd.DataFrame(fq)
    amp = [pd.DataFrame(F2_abs_amp[i])for i in range(0, len(data_label), 1)]
    a = [pd.concat([fq_amp, amp[i]], axis=1, ignore_index=True)for i in range(0, len(data_label), 1)]

    a = [pd.DataFrame(a[i]).drop(a[i].index[[0]])for i in range(0, len(data_label), 1)]
    a = [a[i][a[i][1] == np.max(a[i][1])]for i in range(0, len(data_label), 1)]
    data_record_gyrn_fq_amp_max = pd.concat(a, ignore_index=True)
    data_record_gyrn_fq_amp_max.columns = ["gyrn_max_fq", "gyrn_max_peak"]
    
#    b = pd.concat([data_record_accn_fq_amp_max, data_record_gyrn_fq_amp_max], axis=1, ignore_index=False)
    return data_record_gyrn_fq_amp_max

data2_c_50_record_fq_amp_max = fq_amp_max_order(data2_c_record_1, data2_c_label_50)
data2_c_60_record_fq_amp_max = fq_amp_max_order(data2_c_record_2, data2_c_label_60)
data2_c_70_record_fq_amp_max = fq_amp_max_order(data2_c_record_3, data2_c_label_70)
data2_c_80_record_fq_amp_max = fq_amp_max_order(data2_c_record_4, data2_c_label_80)

# data2_c_50_record_fq_amp_max = fq_amp_max_order(data2_c_record_1, data2_c_label_50)
# data2_c_60_record_fq_amp_max = fq_amp_max_order(data2_c_record_2, data2_c_label_60)
# data2_c_70_record_fq_amp_max = fq_amp_max_order(data2_c_record_3, data2_c_label_70)
# data2_c_80_record_fq_amp_max = fq_amp_max_order(data2_c_record_4, data2_c_label_80)

# a = data2_c_50_record_fq_amp_max
# b = data2_c_60_record_fq_amp_max
# c = data2_c_70_record_fq_amp_max
# d = data2_c_80_record_fq_amp_max

# e = data2_c_50_record_fq_amp_max
# f = data2_c_60_record_fq_amp_max
# g = data2_c_70_record_fq_amp_max
# h = data2_c_80_record_fq_amp_max

# fq_amp_max_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# fq_amp_max_list

In [51]:
#全特徴量の行列の導出
data2_c_50_record_all_feature = pd.concat([data2_c_50_record_mean, data2_c_50_record_var, data2_c_50_record_max, data2_c_50_record_min, data2_c_50_record_fq_amp_max], axis=1, ignore_index=False)
data2_c_60_record_all_feature = pd.concat([data2_c_60_record_mean, data2_c_60_record_var, data2_c_60_record_max, data2_c_60_record_min, data2_c_60_record_fq_amp_max], axis=1, ignore_index=False)
data2_c_70_record_all_feature = pd.concat([data2_c_70_record_mean, data2_c_70_record_var, data2_c_70_record_max, data2_c_70_record_min, data2_c_70_record_fq_amp_max], axis=1, ignore_index=False)
data2_c_80_record_all_feature = pd.concat([data2_c_80_record_mean, data2_c_80_record_var, data2_c_80_record_max, data2_c_80_record_min, data2_c_80_record_fq_amp_max], axis=1, ignore_index=False)

# data2_c_50_record_all_feature = pd.concat([data2_c_50_record_mean, data2_c_50_record_var, data2_c_50_record_max, data2_c_50_record_min, data2_c_50_record_fq_amp_max], axis=1, ignore_index=False)
# data2_c_60_record_all_feature = pd.concat([data2_c_60_record_mean, data2_c_60_record_var, data2_c_60_record_max, data2_c_60_record_min, data2_c_60_record_fq_amp_max], axis=1, ignore_index=False)
# data2_c_70_record_all_feature = pd.concat([data2_c_70_record_mean, data2_c_70_record_var, data2_c_70_record_max, data2_c_70_record_min, data2_c_70_record_fq_amp_max], axis=1, ignore_index=False)
# data2_c_80_record_all_feature = pd.concat([data2_c_80_record_mean, data2_c_80_record_var, data2_c_80_record_max, data2_c_80_record_min, data2_c_80_record_fq_amp_max], axis=1, ignore_index=False)

In [52]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
data2_c_60_record_all_feature

,gyrx_mean,gyry_mean,gyrz_mean,gyrn_mean,gyrx_var,gyry_var,gyrz_var,gyrn_var,gyrx_max,gyry_max,gyrz_max,gyrn_max,gyrx_min,gyry_min,gyrz_min,gyrn_min,gyrn_max_fq,gyrn_max_peak
0,94.003,78.505,-126.450,740.733907,46726.810991,6.955861e+05,27883.575500,2.524989e+05,870,3165,381,3193.702867,-581,-2944,-618,12.449900,1.956947,188.290269
1,83.035,94.897,-155.670,795.291419,48949.915775,8.226027e+05,32368.945100,3.115665e+05,870,3165,381,3193.702867,-581,-2944,-637,12.449900,0.391389,168.164193
2,91.909,86.690,-108.409,839.410801,81297.120719,8.777112e+05,45601.263719,3.277140e+05,1034,3165,637,3193.702867,-678,-2944,-637,12.449900,3.131115,207.500972
3,93.419,74.898,-143.468,1059.124445,122370.671439,1.797995e+06,64680.262976,8.982210e+05,1382,4817,698,8287.917229,-1532,-8194,-1033,51.254268,3.131115,262.200270
4,78.066,33.394,-116.523,982.307334,115358.407644,1.661667e+06,63788.729471,8.966739e+05,1382,4817,698,8287.917229,-1532,-8194,-1033,57.288742,0.782779,280.568331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710,85.194,-144.980,-110.587,1626.430191,443698.918364,6.127942e+06,56716.968431,4.023590e+06,3479,11031,570,22488.584549,-1995,-22389,-1070,26.324893,0.391389,867.250344
711,78.797,-220.082,-87.766,1780.896626,498168.599791,6.508455e+06,66910.313244,3.964289e+06,3479,11031,722,22488.584549,-1995,-22389,-1070,71.721684,1.956947,1293.162325
712,33.529,-76.352,-86.743,1863.676895,552703.057159,6.603601e+06,88690.566951,3.786181e+06,3479,11031,1045,22488.584549,-2367,-22389,-1070,127.291005,1.565558,934.473139
713,82.658,107.140,-85.427,2184.377955,625788.009036,1.005215e+07,106147.602671,6.038187e+06,2589,12555,1045,25284.214740,-2367,-25255,-1576,111.000000,1.565558,534.814149


In [53]:
#各特徴量データの保存
data2_c_50_record_all_feature.to_csv('feature/C/data2_c_50_record_all_feature.csv', index=False)
data2_c_60_record_all_feature.to_csv('feature/C/data2_c_60_record_all_feature.csv', index=False)
data2_c_70_record_all_feature.to_csv('feature/C/data2_c_70_record_all_feature.csv', index=False)
data2_c_80_record_all_feature.to_csv('feature/C/data2_c_80_record_all_feature.csv', index=False)

# data2_c_50_record_all_feature.to_csv('feature/C/data2_c_50_record_all_feature.csv', index=False)
# data2_c_60_record_all_feature.to_csv('feature/C/data2_c_60_record_all_feature.csv', index=False)
# data2_c_70_record_all_feature.to_csv('feature/C/data2_c_70_record_all_feature.csv', index=False)
# data2_c_80_record_all_feature.to_csv('feature/C/data2_c_80_record_all_feature.csv', index=False)

In [54]:
data2_c_label_50

,start_time,finish_time,emotion_level
0,2020-11-11 15:15:24,2020-11-11 15:15:28,2
1,2020-11-11 15:15:25,2020-11-11 15:15:29,2
2,2020-11-11 15:15:26,2020-11-11 15:15:30,2
3,2020-11-11 15:15:27,2020-11-11 15:15:31,2
4,2020-11-11 15:15:28,2020-11-11 15:15:32,2
...,...,...,...
723,2020-11-11 15:29:48,2020-11-11 15:29:52,1
724,2020-11-11 15:29:49,2020-11-11 15:29:53,1
725,2020-11-11 15:29:50,2020-11-11 15:29:54,1
726,2020-11-11 15:29:51,2020-11-11 15:29:55,1


In [55]:
data2_c_label_50_time = data2_c_label_50.drop("emotion_level", axis=1)
data2_c_label_60_time = data2_c_label_60.drop("emotion_level", axis=1)
data2_c_label_70_time = data2_c_label_70.drop("emotion_level", axis=1)
data2_c_label_80_time = data2_c_label_80.drop("emotion_level", axis=1)

# data2_c_label_50_time = data2_c_label_50.drop("emotion_level", axis=1)
# data2_c_label_60_time = data2_c_label_60.drop("emotion_level", axis=1)
# data2_c_label_70_time = data2_c_label_70.drop("emotion_level", axis=1)
# data2_c_label_80_time = data2_c_label_80.drop("emotion_level", axis=1)

In [56]:
data2_c_label_50_time

,start_time,finish_time
0,2020-11-11 15:15:24,2020-11-11 15:15:28
1,2020-11-11 15:15:25,2020-11-11 15:15:29
2,2020-11-11 15:15:26,2020-11-11 15:15:30
3,2020-11-11 15:15:27,2020-11-11 15:15:31
4,2020-11-11 15:15:28,2020-11-11 15:15:32
...,...,...
723,2020-11-11 15:29:48,2020-11-11 15:29:52
724,2020-11-11 15:29:49,2020-11-11 15:29:53
725,2020-11-11 15:29:50,2020-11-11 15:29:54
726,2020-11-11 15:29:51,2020-11-11 15:29:55


In [57]:
data2_c_50_record_all_feature = pd.concat([data2_c_label_50_time, data2_c_50_record_all_feature], axis=1)
data2_c_60_record_all_feature = pd.concat([data2_c_label_60_time, data2_c_60_record_all_feature], axis=1)
data2_c_70_record_all_feature = pd.concat([data2_c_label_70_time, data2_c_70_record_all_feature], axis=1)
data2_c_80_record_all_feature = pd.concat([data2_c_label_80_time, data2_c_80_record_all_feature], axis=1)

# data2_c_50_record_all_feature = pd.concat([data2_c_label_50_time, data2_c_50_record_all_feature], axis=1)
# data2_c_60_record_all_feature = pd.concat([data2_c_label_60_time, data2_c_60_record_all_feature], axis=1)
# data2_c_70_record_all_feature = pd.concat([data2_c_label_70_time, data2_c_70_record_all_feature], axis=1)
# data2_c_80_record_all_feature = pd.concat([data2_c_label_80_time, data2_c_80_record_all_feature], axis=1)

In [58]:
data2_c_60_record_all_feature

,start_time,finish_time,gyrx_mean,gyry_mean,gyrz_mean,gyrn_mean,gyrx_var,gyry_var,gyrz_var,gyrn_var,gyrx_max,gyry_max,gyrz_max,gyrn_max,gyrx_min,gyry_min,gyrz_min,gyrn_min,gyrn_max_fq,gyrn_max_peak
0,2020-11-11 15:15:24,2020-11-11 15:15:28,94.003,78.505,-126.450,740.733907,46726.810991,6.955861e+05,27883.575500,2.524989e+05,870,3165,381,3193.702867,-581,-2944,-618,12.449900,1.956947,188.290269
1,2020-11-11 15:15:25,2020-11-11 15:15:29,83.035,94.897,-155.670,795.291419,48949.915775,8.226027e+05,32368.945100,3.115665e+05,870,3165,381,3193.702867,-581,-2944,-637,12.449900,0.391389,168.164193
2,2020-11-11 15:15:26,2020-11-11 15:15:30,91.909,86.690,-108.409,839.410801,81297.120719,8.777112e+05,45601.263719,3.277140e+05,1034,3165,637,3193.702867,-678,-2944,-637,12.449900,3.131115,207.500972
3,2020-11-11 15:15:27,2020-11-11 15:15:31,93.419,74.898,-143.468,1059.124445,122370.671439,1.797995e+06,64680.262976,8.982210e+05,1382,4817,698,8287.917229,-1532,-8194,-1033,51.254268,3.131115,262.200270
4,2020-11-11 15:15:28,2020-11-11 15:15:32,78.066,33.394,-116.523,982.307334,115358.407644,1.661667e+06,63788.729471,8.966739e+05,1382,4817,698,8287.917229,-1532,-8194,-1033,57.288742,0.782779,280.568331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710,2020-11-11 15:29:48,2020-11-11 15:29:52,85.194,-144.980,-110.587,1626.430191,443698.918364,6.127942e+06,56716.968431,4.023590e+06,3479,11031,570,22488.584549,-1995,-22389,-1070,26.324893,0.391389,867.250344
711,2020-11-11 15:29:49,2020-11-11 15:29:53,78.797,-220.082,-87.766,1780.896626,498168.599791,6.508455e+06,66910.313244,3.964289e+06,3479,11031,722,22488.584549,-1995,-22389,-1070,71.721684,1.956947,1293.162325
712,2020-11-11 15:29:50,2020-11-11 15:29:54,33.529,-76.352,-86.743,1863.676895,552703.057159,6.603601e+06,88690.566951,3.786181e+06,3479,11031,1045,22488.584549,-2367,-22389,-1070,127.291005,1.565558,934.473139
713,2020-11-11 15:29:51,2020-11-11 15:29:55,82.658,107.140,-85.427,2184.377955,625788.009036,1.005215e+07,106147.602671,6.038187e+06,2589,12555,1045,25284.214740,-2367,-25255,-1576,111.000000,1.565558,534.814149


In [59]:
data2_c_50_record_all_feature.to_csv('feature/C/data2_c_50_record_all_feature.csv', index=False)
data2_c_60_record_all_feature.to_csv('feature/C/data2_c_60_record_all_feature.csv', index=False)
data2_c_70_record_all_feature.to_csv('feature/C/data2_c_70_record_all_feature.csv', index=False)
data2_c_80_record_all_feature.to_csv('feature/C/data2_c_80_record_all_feature.csv', index=False)

# data2_c_50_record_all_feature.to_csv('feature/C/data2_c_50_record_all_feature.csv', index=False)
# data2_c_60_record_all_feature.to_csv('feature/C/data2_c_60_record_all_feature.csv', index=False)
# data2_c_70_record_all_feature.to_csv('feature/C/data2_c_70_record_all_feature.csv', index=False)
# data2_c_80_record_all_feature.to_csv('feature/C/data2_c_80_record_all_feature.csv', index=False)

In [60]:
data2_c_60_record_all_feature

,start_time,finish_time,gyrx_mean,gyry_mean,gyrz_mean,gyrn_mean,gyrx_var,gyry_var,gyrz_var,gyrn_var,gyrx_max,gyry_max,gyrz_max,gyrn_max,gyrx_min,gyry_min,gyrz_min,gyrn_min,gyrn_max_fq,gyrn_max_peak
0,2020-11-11 15:15:24,2020-11-11 15:15:28,94.003,78.505,-126.450,740.733907,46726.810991,6.955861e+05,27883.575500,2.524989e+05,870,3165,381,3193.702867,-581,-2944,-618,12.449900,1.956947,188.290269
1,2020-11-11 15:15:25,2020-11-11 15:15:29,83.035,94.897,-155.670,795.291419,48949.915775,8.226027e+05,32368.945100,3.115665e+05,870,3165,381,3193.702867,-581,-2944,-637,12.449900,0.391389,168.164193
2,2020-11-11 15:15:26,2020-11-11 15:15:30,91.909,86.690,-108.409,839.410801,81297.120719,8.777112e+05,45601.263719,3.277140e+05,1034,3165,637,3193.702867,-678,-2944,-637,12.449900,3.131115,207.500972
3,2020-11-11 15:15:27,2020-11-11 15:15:31,93.419,74.898,-143.468,1059.124445,122370.671439,1.797995e+06,64680.262976,8.982210e+05,1382,4817,698,8287.917229,-1532,-8194,-1033,51.254268,3.131115,262.200270
4,2020-11-11 15:15:28,2020-11-11 15:15:32,78.066,33.394,-116.523,982.307334,115358.407644,1.661667e+06,63788.729471,8.966739e+05,1382,4817,698,8287.917229,-1532,-8194,-1033,57.288742,0.782779,280.568331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710,2020-11-11 15:29:48,2020-11-11 15:29:52,85.194,-144.980,-110.587,1626.430191,443698.918364,6.127942e+06,56716.968431,4.023590e+06,3479,11031,570,22488.584549,-1995,-22389,-1070,26.324893,0.391389,867.250344
711,2020-11-11 15:29:49,2020-11-11 15:29:53,78.797,-220.082,-87.766,1780.896626,498168.599791,6.508455e+06,66910.313244,3.964289e+06,3479,11031,722,22488.584549,-1995,-22389,-1070,71.721684,1.956947,1293.162325
712,2020-11-11 15:29:50,2020-11-11 15:29:54,33.529,-76.352,-86.743,1863.676895,552703.057159,6.603601e+06,88690.566951,3.786181e+06,3479,11031,1045,22488.584549,-2367,-22389,-1070,127.291005,1.565558,934.473139
713,2020-11-11 15:29:51,2020-11-11 15:29:55,82.658,107.140,-85.427,2184.377955,625788.009036,1.005215e+07,106147.602671,6.038187e+06,2589,12555,1045,25284.214740,-2367,-25255,-1576,111.000000,1.565558,534.814149
